In [ ]:
import hylite
from hylite import io, HyLibrary
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In addition to image and cloud datasets, it is common to build and work with libraries of individual spectra, either extracted from an image dataset (e.g. endmembers) or collected with portable or laboratory spectrometers (e.g. the USGS spectral library). 

Spectral libraries are represented in *hylite* as special kinds of images, in which each column of pixels represents a sample and each row a single measurement associated with it. Thus, the HyLibrary object contains a data array that is indexed as: ```lib.data[ sample, measurement, band ]```. Note that if some samples contain less measurements than others then some samples may be padded with nan values to ensure the array has a consistent shape.

In the following, we will create a spectral library from an image dataset, and then supplement it with spectra from the USGS library.

### Build a spectral library from scanned samples

In [ ]:
# load image dataset
image = io.load( 'test_data/image.hdr' )
fig,ax=image.quick_plot(hylite.SWIR, figsize=(10,5), ticks=True)
fig.show()

In [ ]:
# label some seed pixels in each sample
image.header.set_sample_points('A', [(50,30)])
image.header.set_sample_points('B', [(150,30)])
image.header.set_sample_points('C', [(225,30)])
image.header['class names'] = ['A','B','C']

In [ ]:
# expand labels using grab-cut
from hylite.filter import label_blocks
cls = label_blocks(image, s=5, # number of pixels to label outside of seed point
                       epad=2,  # ignore these pixels near edges (can be dodgy sometimes)
                       erode=2, # apply erode filter to avoid pixels near sample edges
                       boost = 10, # boost contrast before labelling
                       bands=hylite.SWIR, figsize=(10,5) )

In [ ]:
# convert to spectral library
from hylite.hylibrary import from_classification
lib = from_classification( image, # image to take spectra from
                           cls, # class labels defining each sample
                          ignore=[0], # ignore 0 labels (these are background pixels)
                          subsample=100) # randomly select 100 pixels from each sample

This library can now be quickly plotted for e.g. visual interpretation:

In [ ]:
fig,ax = lib.quick_plot(color=['r','g','b'], # colours for each sample spectra
               clip=(5,50,90), # display median and shaded area between 5th and 90th percentile
               figsize=(8,4)) 
fig.show()

Often it is most informative to plot spectral libraries as hull-corrected spectra. While you can do this manually using the `get_hull_corrected(lib)`, hull correction has also been inbuilt into the quickplot function for convenience:

In [ ]:
fig,ax = lib.quick_plot(color=['r','g','b'], # colours for each sample spectra
               clip=(5,50,90), # display median and shaded area between 5th and 90th percentile
               hc=True, # do hull correction
               figsize=(8,4)) 
fig.show()

### Load reference spectra from USGS library

To help identify the likely minerals in these three samples, we will load some spectra from the USGS library. These can be downloaded in a simple text format from [here](https://crustal.usgs.gov/speclab/QueryAll07a.php), though some spectra have already been included in the example datasets.

We will first load the mineral reflectance spectra and associated wavelengths using numpy:

In [ ]:
ref = []
for m in ['muscovite', 'chlorite']:
    w = np.loadtxt('test_data/minerals/%s_wav.txt'%m, skiprows=1) * 1000.
    r = np.clip( np.loadtxt('test_data/minerals/%s.txt'%m, skiprows=1), 0, 1)
    ref.append( HyLibrary( r[None,None,:], # reshape spectra to [ sample, measurement, bands ]
                          lab=[m], # mineral name
                          wav = w ) )# specify wavelengths
musc,chl = ref
print("Muscovite spectra has %d bands. Chlorite spectra has %d bands" % (musc.band_count(), chl.band_count()))

You will notice that the muscovite and chlorite spectra here were sampled using different instruments and so have different spectral resolution. As a result, we need to resample them so that they can be combined into a single spectral library. The `HyData.resample` function makes this quite easy; to keep things simple, we will resample both libraries onto the same wavelength array as our sample spectra to allow direct comparisons.

In [ ]:
# resample and combine HyLibraries
ref = musc.resample( lib.get_wavelengths(), vb=False ) + chl.resample( lib.get_wavelengths(), vb=False )
fig,ax = ref.quick_plot(hc=True, figsize=(8,4))
fig.show()

To finish off, we'll build a nice figure that compares our sample spectra with the reference ones. Can you tell which samples contain which minerals? ¯\_(ツ)_/¯

In [ ]:
fig,ax = plt.subplots(2,2, figsize=(10,6))

ax[0,0].set_title('a. Measured spectra')
lib.quick_plot(ax=ax[0,0], color='k' )

ax[0,1].set_title('b. Measured spectra (hull corrected)')
lib.quick_plot(ax=ax[0,1], hc=True, color='k')

ax[1,0].set_title('c. Reference minerals')
ref.quick_plot(ax=ax[1,0], color=['gold', 'green'])

ax[1,1].set_title('d. Reference minerals (hull corrected)')
ref.quick_plot(ax=ax[1,1], hc=True, color=['gold', 'green'])

fig.tight_layout()
fig.show()
